In [1]:
!pwd

/Users/Bingumalla Likith/Desktop/MLOPS/Project-2/research


In [2]:
import os
os.chdir('../')

In [3]:
!pwd

/Users/Bingumalla Likith/Desktop/MLOPS/Project-2


In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from src.data_science.constants import *
from src.data_science.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self,
                config_path = CONFIG_FILE_PATH,
                params_path = PARAMS_FILE_PATH,
                schema_path = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        self.schema = read_yaml(schema_path)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(**config)
        return data_ingestion_config

In [6]:
## Component-1 => DataIngestion
import os
import urllib.request as request
from src.data_science import logger
import zipfile

class DataIngestion:
    def __init__(self, config : DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename , headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists")
    
    def extract_zip_file(self):
        '''Extract the data from the zip file'''
        unzip_dir = self.config.unzip_dir
        os.makedirs(unzip_dir, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file , 'r') as file:
            file.extractall(unzip_dir)

In [7]:
try:
    config = ConfigurationManager()
    data_ingest_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingest_config)

    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-12-29 18:33:44,509 : INFO : common : Yaml file : config/config.yaml loaded successfully !!]
[2024-12-29 18:33:44,510 : INFO : common : Yaml file : params.yaml loaded successfully !!]
[2024-12-29 18:33:44,510 : INFO : common : Yaml file : schema.yaml loaded successfully !!]
[2024-12-29 18:33:44,511 : INFO : common : Created directory at : artifacts]
[2024-12-29 18:33:44,511 : INFO : common : Created directory at : artifacts/data_ingestion]
[2024-12-29 18:33:45,571 : INFO : 1369555160 : artifacts/data_ingestion/data.zip download! with following info: 
X-GUploader-UploadID: AFiumC57YgaZtNHIiXwxh1-T5Zf3Uzg8ZHidOtuAb3ZIbmwLTsvlxl745JM5zyJcb_4m5JvOoPJ5u_c
Expires: Sun, 29 Dec 2024 13:03:45 GMT
Date: Sun, 29 Dec 2024 13:03:45 GMT
Cache-Control: private, max-age=0
Last-Modified: Fri, 13 Dec 2024 09:29:19 GMT
ETag: "d2c57f6b2d1976ddc86f11b70fc65bb5"
x-goog-generation: 1734082159542247
x-goog-metageneration: 1
x-goog-stored-content-encoding: identity
x-goog-stored-content-length: 20027
Con